### Задача
Считать таблицу из credit_train.csv. Построить модель предсказывающую целевую переменную open_account_flg. Задача является задачей бинарной классификации

In [ ]:
!pip install pandas numpy matplotlib sklearn_pandas;

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_orig = pd.read_csv('./credit_train.csv', sep=';', index_col='client_id')

df = df_orig.copy(deep=True)
target = df['open_account_flg']
df.columns

Index(['gender', 'age', 'marital_status', 'job_position', 'credit_sum',
       'credit_month', 'tariff_id', 'score_shk', 'education', 'living_region',
       'monthly_income', 'credit_count', 'overdue_credit_count',
       'open_account_flg'],
      dtype='object')

In [4]:
from sklearn_pandas import CategoricalImputer
import sanitizer

df = sanitizer.sanitize_frame(df)

In [6]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics

X = df.drop(columns=['open_account_flg'])
y = df['open_account_flg']

cv=8
max_iter=2500
n_jobs=-1
random_state=42

clf_rocauc = LogisticRegressionCV(max_iter=max_iter, cv=cv, scoring='roc_auc', n_jobs=n_jobs, random_state=random_state)
clf_rocauc.fit(X, y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=8, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=2500, multi_class='warn', n_jobs=-1, penalty='l2',
                     random_state=42, refit=True, scoring='roc_auc',
                     solver='lbfgs', tol=0.0001, verbose=0)

In [13]:
print ('Max roc_auc: {}'.format(clf_rocauc.scores_[1].max()))

Max roc_auc: 0.741466080662243


In [9]:
clf_f1 = LogisticRegressionCV(max_iter=max_iter, cv=cv, scoring='f1', n_jobs=n_jobs, random_state=random_state)
clf_f1.fit(X, y)

LogisticRegressionCV(Cs=10, class_weight=None, cv=8, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=2500, multi_class='warn', n_jobs=-1, penalty='l2',
                     random_state=42, refit=True, scoring='f1', solver='lbfgs',
                     tol=0.0001, verbose=0)

In [10]:
print ('Max f1: {}'.format(clf_f1.scores_[1].max()))

Max f1: 0.148832909637162


#### Подготовка
Для подготовки модели были произведены следующие действия:
 - Обработка категориальных признаков(через one hot encoding)
 - Нормализация некоторых непрерывных признаков(monthly_income и т.д.)
 - Добавление новых признаков на основе одного или нескольких(credit_history_ratio, dist и т.д.)
 - Приведение к нормальной форме living_regions. Столбец довольно грязный, поэтому использовался yandex geocoder API для получения "чистых" локаций и их координат
 - Удаление несущественных признаков(gender)
 
#### Обучение
После подготовки данных модель обучалась на 8 фолдах используя логистическую регрессию с метриками roc_auc и f1. Принципиально не использовались другие алгоритмы, интересно было посмотреть именно на перфоманс логистической регрессии

#### Оценка
 - Из-за не очень сбалансированных классов решил не использовать метрику accuracy. Так как она показывала не очень стабильные результаты(оно и ясно, в принципе, из описания данной метрики)
 - Попробовал измерить скор модели на двух метриках - roc_auc и f1. Результаты сверху. По rocauc получаем более менее приемлимые 74 процента. По f1 получились уже скудненькие 14.8%. Мои предположения, что логистическая регрессия для данной задачи не самый оптимальный вариант, поэтому такие вот результаты

#### Что можно сделать для увеличения скора:
 - Классы не очень несбалансированы, попробовать применить алгоритм SMOTE
 - Есть довольно неплохое решение - добавлять smoothed likelihood для каждого категориального столбца. После этого добавлять такую же переменную для пар столбцов, потом для троек и т.д. и т.д. После применить двойную кросс валидацию(double cross-validation). Он в свое время занял призовое место по данному соревнованию
 $$ Smoothed Likelihood = \frac{mean(target) + globalmean+\alpha}{nrows+\alpha} $$
Где globalmean - среднее значение целевой переменной по всей выборке, alpha - коэффициент регуляризации
 - Попробовать другие алгоритмы для обучения
